In [3]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()
#Todo esto es la inicializacion, siempre necesitaremos uns parkcontext desde el cual empezaremos a trabajar



# keys()

In [7]:
#Hacemos un nuevo RDD de pares por ejemplo usando ZIP que hemos aprendido antes a usarlo

rdd_Clave_valor = sc.parallelize(zip(['a','b','c','d','e'],range(5)))
rdd_Clave_valor.collect()

[('a', 0), ('b', 1), ('c', 2), ('d', 3), ('e', 4)]

In [8]:
rdd_Clave_valor.keys().collect()

['a', 'b', 'c', 'd', 'e']

In [9]:
#Simplemente nos devuelve las claves de nuestro RDD

# values()

In [10]:
rdd_Clave_valor.values().collect()

[0, 1, 2, 3, 4]

In [11]:
#Similar al anterior, pero en este caso nos va a devolver un rdd con los valores

# sortByKey()

In [6]:
rdd_Desordenado = sc.parallelize(([('a',1),('z',3),('b',4),('c',3)]))

In [7]:
rdd_Desordenado.collect()

[('a', 1), ('z', 3), ('b', 4), ('c', 3)]

In [8]:
rdd_Desordenado.sortByKey().collect()

[('a', 1), ('b', 4), ('c', 3), ('z', 3)]

In [9]:
# Vemos que nos lo ha ordenado alfabeticamente si tuvieramos elementos con la misma clave, los podria juntos
rdd_Desordenado = sc.parallelize(([('a',1),('z',3),('b',4),('c',3),('a',4)]))

In [10]:
rdd_Desordenado.sortByKey().collect()

[('a', 1), ('a', 4), ('b', 4), ('c', 3), ('z', 3)]

In [11]:
#Si indicamos  False  nos ordenará descendente:
rdd_Desordenado.sortByKey(False).collect()

[('z', 3), ('c', 3), ('b', 4), ('a', 1), ('a', 4)]

In [12]:
#Si indicamos  True  nos ordenará ascendente, igual que si no le decimos nada:

rdd_Desordenado.sortByKey(True).collect()

[('a', 1), ('a', 4), ('b', 4), ('c', 3), ('z', 3)]

# mapValues()


In [17]:
#Aplica una funcion sobre los valores

In [15]:
rdd= sc.parallelize(([('a',1),('z',3),('b',4),('c',3),('a',4)]))

In [19]:
rdd_map = rdd.mapValues(lambda x: x*2)

In [20]:
rdd_map.collect()
#Como vemos solo actua sobre los valores

[('a', 2), ('z', 6), ('b', 8), ('c', 6), ('a', 8)]

In [21]:
#Hemos multiplado cada valor por 2 y lo devuelve en un RDD
#Podriamos devolver otra tupla dentro de la tupla, por ejemplo:

rdd_map = rdd.mapValues(lambda x: (x,x*2))
rdd_map.collect()

[('a', (1, 2)), ('z', (3, 6)), ('b', (4, 8)), ('c', (3, 6)), ('a', (4, 8))]

In [22]:
#Podemos devolver los tipos de datos que queramos
rdd_map = rdd.mapValues(lambda x: ('valor antes:'+str(x),'valor *2:'+str(x*2)))

In [23]:
rdd_map.collect()

[('a', ('valor antes:1', 'valor *2:2')),
 ('z', ('valor antes:3', 'valor *2:6')),
 ('b', ('valor antes:4', 'valor *2:8')),
 ('c', ('valor antes:3', 'valor *2:6')),
 ('a', ('valor antes:4', 'valor *2:8'))]

# flatMapValues()

In [24]:
#Crea una lista simple 

In [16]:
rdd_flatmap = rdd.flatMapValues(lambda x: (x,x*2))

In [17]:
rdd_flatmap.collect()

[('a', 1),
 ('a', 2),
 ('z', 3),
 ('z', 6),
 ('b', 4),
 ('b', 8),
 ('c', 3),
 ('c', 6),
 ('a', 4),
 ('a', 8)]

In [20]:
#Si vemos la difencia con mapValues, vemos que flatmapvalues nos devuelve una lista aplanada, y mapvalues lo agrupa
rdd_flatmap = rdd.mapValues(lambda x: (x,x*2))
rdd_flatmap.collect()

[('a', (1, 2)), ('z', (3, 6)), ('b', (4, 8)), ('c', (3, 6)), ('a', (4, 8))]

In [ ]:
#si tuvieramos un rdd con una clave y una lista por valor, con flatmapvalues podriamos separar esta lista con su valor

In [30]:
ejemplo_paraflat = sc.parallelize([("a", ["x", "y", "z"]), ("b", ["p", "r"])])

In [33]:
ejemplo_paraflat.flatMapValues(lambda x:x).collect()

[('a', 'x'), ('a', 'y'), ('a', 'z'), ('b', 'p'), ('b', 'r')]

In [ ]:
#Como vemos aplana la lista y nos devuelve una lista simple

# subtractByKey()

Dados dos RDD de pares, elimina de uno los elementos cuya clave aparece en el otro rdd


In [22]:
RDD1 = sc.parallelize([('a',1),('a',3),('b',1),('c',1)])
RDD2 = sc.parallelize([('a',1),('r',3),('t',1),('x',1)])


In [23]:
RDD1.subtractByKey(RDD2).collect()

[('b', 1), ('c', 1)]

In [24]:
#Como en RDD2 tnemos la key 'a' nos borra del 1 todas las que tengan esa clave

# cogroup

agrupa por claves, y devuelve una lista iterable por cada RDD y por cada una de las claves


In [38]:
cogrupo_ejemplo = RDD1.cogroup(RDD2)

In [39]:
cogrupo_ejemplo.collect()
#Como vemos el resultado es la clave y dos iterables como valor

[('b',
  (<pyspark.resultiterable.ResultIterable at 0x255ac5938b0>,
   <pyspark.resultiterable.ResultIterable at 0x255ac59b190>)),
 ('c',
  (<pyspark.resultiterable.ResultIterable at 0x255ac59b1f0>,
   <pyspark.resultiterable.ResultIterable at 0x255ac59b250>)),
 ('x',
  (<pyspark.resultiterable.ResultIterable at 0x255ac59b2b0>,
   <pyspark.resultiterable.ResultIterable at 0x255ac59b310>)),
 ('r',
  (<pyspark.resultiterable.ResultIterable at 0x255ac59b370>,
   <pyspark.resultiterable.ResultIterable at 0x255ac59b3d0>)),
 ('a',
  (<pyspark.resultiterable.ResultIterable at 0x255ac59b430>,
   <pyspark.resultiterable.ResultIterable at 0x255ac59b4c0>)),
 ('t',
  (<pyspark.resultiterable.ResultIterable at 0x255ac59b520>,
   <pyspark.resultiterable.ResultIterable at 0x255ac59b580>))]

In [40]:
#Si queremos recorrer esto es un poco mas complejo
#seria algo asi:

[(x, tuple(map(list, y))) for x, y in sorted(list(cogrupo_ejemplo.collect()))]


[('a', ([1, 3], [1])),
 ('b', ([1], [])),
 ('c', ([1], [])),
 ('r', ([], [3])),
 ('t', ([], [1])),
 ('x', ([], [1]))]